## Постановка задачи
Загрузите данные, приведите их к числовым, заполните пропуски, нормализуйте данные и оптимизируйте память.

Сформируйте параллельный ансамбль из CatBoost, градиентного бустинга, XGBoost и LightGBM. Используйте лучшие гиперпараметры, подобранные ранее, или найдите их через перекрестную проверку. Итоговое решение рассчитайте на основании самого точного предсказания класса у определенной модели ансамбля: выберите для каждого класса модель, которая предсказывает его лучше всего.

Проведите расчеты и выгрузите результат в виде submission.csv

Данные:
* video.ittensive.com/machine-learning/prudential/train.csv.gz
* video.ittensive.com/machine-learning/prudential/test.csv.gz
* video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz



### Подключение библиотек

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn import preprocessing
from xgboost import XGBClassifier
from catboost import Pool, CatBoostClassifier

### Загрузка данных

In [34]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [35]:
def data_preprocess (df):
    df["Product_Info_2_1"] = df["Product_Info_2"].str.slice(0, 1)
    df["Product_Info_2_2"] = pd.to_numeric(df["Product_Info_2"].str.slice(1, 2))
    df.drop(labels=["Product_Info_2"], axis=1, inplace=True)
    for l in df["Product_Info_2_1"].unique():
        df["Product_Info_2_1" + l] = df["Product_Info_2_1"].isin([l]).astype("int8")
    df.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
    df.fillna(value=-1, inplace=True)
    data["Response"] = data["Response"] - 1
    return df

In [36]:
data = data_preprocess(data)

### Набор столбцов для расчета

In [37]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [38]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [39]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [40]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [41]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
18084  0.191284  1.249023 -1.905273 -0.479736  0.611816 -0.169434 -1.159180   
32882  0.426270  1.494141  0.213989 -0.361572  0.611816 -0.169434  0.862305   
32433  1.177734  1.494141 -0.164429  0.432861  0.611816 -0.169434  0.862305   
57304 -1.429688 -0.465576 -0.164429 -1.550781  0.611816 -0.169434 -1.159180   
54181  0.449707  1.003906  0.365479 -0.055756 -1.634766 -0.169434  0.862305   

              7         8         9  ...       109      110       111  \
18084  1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   
32882  0.043671  0.862305 -0.928711  ... -0.083679  0.44165 -0.149292   
32433 -1.013672  0.867676 -0.928711  ... -0.083679  0.44165 -0.149292   
57304  1.100586 -1.156250  1.130859  ... -0.083679  0.44165 -0.149292   
54181 -1.013672  0.861816 -0.928711  ... -0.083679  0.44165 -0.149292   

            112       113       114       115       116      117  Response  
18084 -0.

### Построение базовых моделей

In [42]:
x = pd.DataFrame(data_transformed, columns=columns_transformed)

LightGBM, используем multiclass

In [43]:
model_lgb = lgb.LGBMRegressor(random_state=17, max_depth=18,
            min_child_samples=17, num_leaves=35, n_estimators=10000)
model_lgb.fit(x, data["Response"])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023428 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2467
[LightGBM] [Info] Number of data points in the train set: 59381, number of used features: 118
[LightGBM] [Info] Start training from score 4.636837


LGBMRegressor(max_depth=18, min_child_samples=17, n_estimators=10000,
              num_leaves=35, random_state=17)

CatBoost

In [46]:
train_dataset = Pool(data=x, label=data["Response"])


In [48]:
model_cb = CatBoostClassifier(iterations=1000, learning_rate=0.57,
            random_seed=17, depth=6, l2_leaf_reg=2,
            loss_function='MultiClass', bootstrap_type="MVS")
model_cb.fit(Pool(data=x, label=data["Response"]))

0:	learn: 1.5330128	total: 91.3ms	remaining: 1m 31s
1:	learn: 1.4359398	total: 176ms	remaining: 1m 27s
2:	learn: 1.3427639	total: 260ms	remaining: 1m 26s
3:	learn: 1.3129551	total: 357ms	remaining: 1m 28s
4:	learn: 1.2920150	total: 447ms	remaining: 1m 28s
5:	learn: 1.2754757	total: 572ms	remaining: 1m 34s
6:	learn: 1.2629586	total: 653ms	remaining: 1m 32s
7:	learn: 1.2538166	total: 737ms	remaining: 1m 31s
8:	learn: 1.2443132	total: 824ms	remaining: 1m 30s
9:	learn: 1.2374051	total: 912ms	remaining: 1m 30s
10:	learn: 1.2335046	total: 992ms	remaining: 1m 29s
11:	learn: 1.2294222	total: 1.1s	remaining: 1m 30s
12:	learn: 1.2236002	total: 1.19s	remaining: 1m 30s
13:	learn: 1.2143246	total: 1.29s	remaining: 1m 30s
14:	learn: 1.2118135	total: 1.37s	remaining: 1m 30s
15:	learn: 1.2085828	total: 1.46s	remaining: 1m 29s
16:	learn: 1.2040890	total: 1.54s	remaining: 1m 29s
17:	learn: 1.1996522	total: 1.67s	remaining: 1m 30s
18:	learn: 1.1974595	total: 1.77s	remaining: 1m 31s
19:	learn: 1.1903650	t

### градиентный бустинг

In [49]:
model_gb = GradientBoostingClassifier(random_state=17, max_depth=13,
                max_features=26, min_samples_leaf=21, n_estimators=75)

In [50]:
model_gb.fit(x, data['Response'])

GradientBoostingClassifier(max_depth=13, max_features=26, min_samples_leaf=21,
                           n_estimators=75, random_state=17)

XGBoost

In [51]:
model_xgb = XGBClassifier(max_depth=17, max_features=27,
                     n_estimators=76, min_samples_leaf=20)

In [52]:
model_xgb.fit(x, data["Response"])

[23:49:03] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "max_features", "min_samples_leaf" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=17, max_features=27,
              max_leaves=None, min_child_weight=None, min_samples_leaf=20,
              missing=nan, monotone_constraints=None, n_estimators=76,
              n_jobs=None, num_parallel_tree=None, ...)

### Загрузка данных для расчета

In [53]:
data_test = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/test.csv.gz")
data_test = data_preprocess(data_test)
data_test = reduce_mem_usage(data_test)
data_test_transformed = pd.DataFrame(scaler.transform(pd.DataFrame(data_test,
                                                     columns=columns)))
print (data_test_transformed.info())

Потребление памяти меньше на 16.34 Мб (минус 84.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19765 entries, 0 to 19764
Columns: 118 entries, 0 to 117
dtypes: float32(118)
memory usage: 8.9 MB
None


### Расчет предказаний

In [54]:
data_test["target_xgb"] = model_xgb.predict(data_test_transformed)

C:\Users\divin\AppData\Local\Temp\ipykernel_10300\1008409102.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_xgb"] = model_xgb.predict(data_test_transformed)


In [55]:
data_test["target_cb"] = model_cb.predict(Pool(data=data_test_transformed))

C:\Users\divin\AppData\Local\Temp\ipykernel_10300\1488459043.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_cb"] = model_cb.predict(Pool(data=data_test_transformed))


In [56]:
data_test["target_gb"] = model_gb.predict(data_test_transformed)

C:\Users\divin\AppData\Local\Temp\ipykernel_10300\1609845201.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_gb"] = model_gb.predict(data_test_transformed)


In [57]:
data_test["target_lgb"] = np.round(model_lgb.predict(data_test_transformed)).astype("int8")

C:\Users\divin\AppData\Local\Temp\ipykernel_10300\2769997431.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_test["target_lgb"] = np.round(model_lgb.predict(data_test_transformed)).astype("int8")


In [60]:
def vote_class (x):
    if x.target_xgb == 2:
        class_ = x.target_xgb
#    elif x.target_lgb == 7:
#        class_ = x.target_lgb
#    elif x.target_cb == 0:
#        class_ = x.target_cb
    else:
        class_ = x.target_gb
    x["Response"] = class_ + 1
    return x

In [61]:
data_test = data_test.apply(vote_class, axis=1)
print (data_test.head())

     Id  Product_Info_1  Product_Info_3  Product_Info_4  Product_Info_5  \
0   1.0             1.0            26.0        0.487061             2.0   
1   3.0             1.0            26.0        0.076904             2.0   
2   4.0             1.0            26.0        0.144653             2.0   
3   9.0             1.0            26.0        0.151733             2.0   
4  12.0             1.0            26.0        0.076904             2.0   

   Product_Info_6  Product_Info_7   Ins_Age        Ht        Wt  ...  \
0             3.0             1.0  0.611816  0.781738  0.338867  ...   
1             3.0             1.0  0.626953  0.727051  0.311768  ...   
2             3.0             1.0  0.582031  0.708984  0.320068  ...   
3             1.0             1.0  0.522461  0.654785  0.267822  ...   
4             3.0             1.0  0.298584  0.672852  0.246826  ...   

   Product_Info_2_1D  Product_Info_2_1A  Product_Info_2_1E  Product_Info_2_1B  \
0                1.0               

### Формирование и выгрузка результатов

In [62]:
submission = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/sample_submission.csv.gz")
submission["Response"] = data_test["Response"].astype("int8")
submission.to_csv("submission.csv", index=False)
print (len(submission["Response"]) + 1)

19766


### Самопроверка модели

In [67]:
data_copy = data_transformed.copy()
x_copy = pd.DataFrame(data_copy, columns=columns_transformed)
copy_dataset = Pool(data=x_copy, label=data_copy["Response"])
data_copy["target_xgb"] = model_xgb.predict(x_copy)
data_copy["target_cb"] = model_cb.predict(copy_dataset)
data_copy["target_gbc"] = model_gb.predict(x_copy)
data_copy["target_lgb"] = np.round(model_lgb.predict(x_copy)).astype("int8")

In [68]:
class_target = ["target_gbc"]*8
def vote_class_enumerate (x):
    for _,target in enumerate(class_target):
        if x[target] == _:
            x["Response"] = x[target]
            break
    return x

In [69]:
kappa_min = 0
for target_model in ["xgb", "cb", "gbc", "lgb"]:
    print ("Проверяем модель:", target_model)
    target_model = "target_" + target_model
    for class_ in range(0,8):
        target_model_prev = class_target[class_]
        class_target[class_] = target_model
        data_copy = data_copy.apply(vote_class_enumerate, axis=1)
        kappa = cohen_kappa_score(data_copy["Response"], 
                data["Response"], weights='quadratic')
        if kappa > kappa_min:
            kappa_min = kappa
        else:
            class_target[class_] = target_model_prev
    print ("Максимальная оценка:", kappa_min)
print (class_target)

Проверяем модель: xgb
Максимальная оценка: 0.91923523748714
Проверяем модель: cb
Максимальная оценка: 0.9192410196373246
Проверяем модель: gbc
Максимальная оценка: 0.9192410196373246
Проверяем модель: lgb
Максимальная оценка: 0.9199970838306729
['target_xgb', 'target_xgb', 'target_cb', 'target_xgb', 'target_xgb', 'target_xgb', 'target_lgb', 'target_xgb']


In [70]:
data_copy = data_copy.apply(vote_class_enumerate, axis=1)

In [71]:
print ("Результат:",
       round(cohen_kappa_score(data_copy["Response"],
                    data["Response"], weights='quadratic'), 3))
print (confusion_matrix(data_copy["Response"], data["Response"]))


Результат: 0.92
[[    0     0     0     0     0     0     0     0     0]
 [ 6170     0     0     0     0     0     0     0     0]
 [    0  6536     0     0     0     0     0     0     0]
 [    0     0   984     4     0     1     0     0     0]
 [    0     0    20  1424     0     0     0     0     0]
 [    0     0     3     0  5432     0     0     0     0]
 [    5     0     1     0     0 11199   207     0     0]
 [    2     0     5     0     0    23  7801   868     0]
 [   30    16     0     0     0    10    19 18621     0]]
